In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
data_contratos = '12/04/2023'

In [4]:
url = f'''
https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/SistemaPregao1.asp?
pagetype=pop&caminho=Resumo%20Estat%EDstico%20-%20Sistema%20Preg%E3o&Data={data_contratos}&Mercadoria=WDO
'''

In [5]:
driver.get(url)

In [8]:
local_tabela = '''
/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[3]/table
'''
local_indice = '''
/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[1]/table
'''

In [9]:
elemento = driver.find_element("xpath", local_tabela)
elemento_indice = driver.find_element("xpath", local_indice)

html_tabela = elemento.get_attribute('outerHTML')
html_indice = elemento_indice.get_attribute('outerHTML')

tabela = pd.read_html(html_tabela)[0]
indice = pd.read_html(html_indice)[0]

driver.quit()

/tmp/ipykernel_5381/2357078203.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabela = pd.read_html(html_tabela)[0]
/tmp/ipykernel_5381/2357078203.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  indice = pd.read_html(html_indice)[0]


In [10]:
indice

,0
0,VENCTO
1,K23
2,M23
3,N23
4,Q23
5,U23
6,V23
7,Z23
8,F24
9,G24


In [11]:
tabela.columns = tabela.loc[0]
tabela = tabela['AJUSTE']
tabela = tabela.drop(0, axis = 0)
indice.columns = indice.loc[0]
indice_dolar = indice['VENCTO']
indice = indice.drop(0, axis = 0)
tabela.index = indice['VENCTO']
tabela = tabela[tabela != "0"]

print(tabela)

VENCTO
K23    4.949,5210
M23    4.980,1650
N23    5.007,7270
Q23    5.034,2340
U23    5.065,3330
V23    5.088,7210
Z23    5.137,0140
F24    5.157,6170
G24    5.184,1230
H24    5.204,2320
V24    5.376,6890
F25    5.456,2780
Name: AJUSTE, dtype: object


In [12]:
legenda = pd.Series(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                    index = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z'])

lista_datas = []

for indice in tabela.index:

    letra = indice[0]
    ano = indice[1:3]

    mes = legenda[letra]

    data = f"{mes}-{ano}"

    data = datetime.strptime(data, "%b-%y")

    lista_datas.append(data)


tabela.index = lista_datas
tabela

2023-05-01    4.949,5210
2023-06-01    4.980,1650
2023-07-01    5.007,7270
2023-08-01    5.034,2340
2023-09-01    5.065,3330
2023-10-01    5.088,7210
2023-12-01    5.137,0140
2024-01-01    5.157,6170
2024-02-01    5.184,1230
2024-03-01    5.204,2320
2024-10-01    5.376,6890
2025-01-01    5.456,2780
Name: AJUSTE, dtype: object